<a href="https://colab.research.google.com/github/LukeTheGeneWriter/AutoClipper/blob/main/Autoclipper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Autoclipper -- Luke Williams 1/7/2025

Near-turn key usage: just fill out the gene dictionary for the virus or other organism you are tracking. Here's the simple breakdown;

Input: Directory of genomes and a dictionary of gene names with example sequences.
Computation: Clips out genes from complete and partial genomes based on sequence homolgy to reference genes.
Output: A file system organized by gene that contains a fasta file for each strain. The file contains all gene snippets for that strain. OutputDir > GeneNames > strain_name.fasta

Tips: If you want to include the gene's promoter, make sure that is in the example dictionary.

Install packages

In [ ]:
#Env
!pip install -q condacolab
import condacolab
condacolab.install()

In [ ]:
!pip install biopython
!conda install -c bioconda pymummer

Imports and file system mount

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount = False)
%ls
%cd drive
%cd MyDrive
%ls

import os
import numpy as np
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
import matplotlib.pyplot as plt
import seaborn as sns
import subprocess
from dataclasses import dataclass
from pymummer import coords_file, alignment, nucmer

#Change these values to run!

#Where do you want to construct the file system?
dest_dir = "Genes_By_Strain"

#Where are the genomes you are working with?
genomes_dir = os.path.join("X", "Y", "Z")

#What is a good threshold value for the complete length of a genome?
length_of_complete_genome = 150000
#
#Thanks! Now just scroll down to the gene dict and fill in your genes.

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
'=1.2.0'                  GeneClasses.py                  md5sum.txt
'=2.16.0'                 GeneLit/                        muscle-linux-x86.v5.3
'=2.2.0'                  Genewriter.AI.gdoc              prepData.ipynb
 301Proj/                 HerpetiQRmiRNAProposals.ipynb   __pycache__/
'=3.7,'                   HerpetiQRpHMM.ipynb             README.md
'Colab Notebooks'/        HSV1AllelesHMM/                 RefGenes/
 condacolab_install.log   HSV1_Genomes/                   secondPassPlusEnergetics/
 DataForWill/             HSV1_Genomes_Strain/            WorkingFolders/
 exonTraining/            HSVReferenceGenome/
 firstPassPlusSpliceAI/   intermediate/
[Errno 2] No such file or directory: 'drive'
/content/drive/MyDrive
[Errno 2] No such file or directory: 'MyDrive'
/content/drive/MyDrive
'=1.2.0'                  GeneClasses.py                  md

Functions - HSV Gene Template

In [ ]:
@dataclass
class MUMmerDelta:
  #[S1]    [E1]    [S2]     [E2]     [LEN 1]   [LEN 2]   [% IDY]   [LEN R]   [LEN Q]   [COV R]   [COV Q]   [TAGS]
  s1: int
  e1: int
  s2: int
  e2: int
  len1: int
  len2: int
  pct_idy: float
  len_r: int
  len_q: int
  idk: int
  refname: str
  queryname: str

In [ ]:
# !!!!!!!!!!
# Replace these example values. Make the table as big or as little as you like.

def getGeneDict():
  #Auto annotater
  gene0 = "AGCCCGGGCCCCGCGCGGCGGGCGTGGGGGGCGAGGAGCGGGGGGGAGGACGGCCGAGAGGCGCGGAGCGCCCGCCCACCCCCCCCGGCCGCCCTTCCCGCTTCCGGCAATTCCCGCGGCCCTTAATGGAGAGGGG"
  gene1 = "ATGGGGATTTTGAACCGAATATGTTATTCGGAGGAGACCCCCTCCCTTTTGCCGCCAGCGCCCCGCCACAACCACTCCTCGTCGATGAATAAA"
  gene2 = "ATGAAGCGGGCCCCGCGACGGGACGCCGCCACACCGGCCGCCTGGGGGGGCTGGCCGAAGTTTAAAGAACTGAGCGGCGGTGGGGGCTTTTGGGTATTTCCGATGAATAAA"
  gene3 = "ATGTCGGGCGTCCGACACGCCAAACCACGAGTCGGCCCGGAACGACGAATGGGGCGTTCCGGGCCGCCTCCGGCCTCGGGGGCCGAGTAACCGCCCCCCCCCCATGCCACCCTCACTGCCCGTCGCGCGTGTTTGATGTTAATAAA"
  gene4 = "TTTAATGGACCGTCCTAGGACCCCAAAAGTTTGCGGCCGTCCCCCGCGGTTGCTGTCCCTCAAAGGCATAGTGTGGATTGGACAT"
  gene5 = "TTTAATGGACCGTCCTAGGACCCCAAAAGTTTGCGGCCGTCCCCCGCGGTTGCTGTCCCTCAAAGGCATATGCTTAAAATCGGGCCGGGTTTCTGTCCGTCTAGCTGGCGCTCCCCGCCGGCCGCCGCCAT"
  gene6 = "ATGACCGCACCACGAGGACGGCTGGGTAAAGGTAACGTCGTCCGCCCGCGGCCCACGGGGCAGCGGGTTTCAGCTTTCAGCACCCGTACATTAGGAAATAAA"
  gene7 = "ATGGCCGCCGCGCCGCAGCCTGGTCGAGGCGGCGCCACCAGCACAGTCCATTTGAGCTTCTGGGCTGCCCCCCTGCTGTTGTTGAATTTTAGGAAATAAA"
  gene8 = "TTTATTGGTCAACTCGCCCCATACTCCCGCCAGGGCCGCGGGTGGGGGCAGCGTCCTCCCTGGCAGCCGGAGGCTTGTCCAT"
  gene9 = "TTTATTGGTCAACTCGCCCCATACTCCCGCCAGGGCCGCGGGTGGGGGCAGCGTCCTCCCTGGCAGCCGGAGGCTTGTCCATCGCACGGCCCCTAAGGA"

  gene_dict = {
    "gene0": gene0,    "gene1": gene1,    "gene2": gene2,    "gene3": gene3,
    "gene4": gene4,    "gene5": gene5,    "gene6": gene6,    "gene7": gene7,
    "gene8": gene8,    "gene9": gene9
    }
  return gene_dict

Functions - automatic genome eater and classifier

In [ ]:
def autoClassifierPyMUMmer(geneRead, genedict, dest = dest_dir, inter_dir = "intermediate", lenpct_thresh = 0.85, id_thresh = 85):
  if not os.path.exists(inter_dir):
    os.makedirs(inter_dir)
  results_file = os.path.join(inter_dir, "results.delta")
  #Write the ref fasta
  reference_file = os.path.join(inter_dir, "reference.fasta")
  with open(reference_file, "w") as f:
    SeqIO.write(geneRead[1], f, "fasta")
  ret_dict = {k:[] for k in genedict.keys()}
  best_classes = {}
  for k in HSV1genedict.keys():
    klen = len(genedict[k])
    print("$$   ", k, " klen: ", klen)
    qrec = SeqRecord(seq=Seq(genedict[k]), id=k, name="Temp_Query_For_Autoclassifier_"+k, description="See Name")
    #Write the query fasta
    query_file = os.path.join(inter_dir, "query.fasta")
    with open(query_file, "w") as f:
      SeqIO.write(qrec, f, "fasta")
    runner = nucmer.Runner(reference_file, query_file, results_file)
    runner.run()
    file_reader = coords_file.reader(results_file)
    alignments = [coord for coord in file_reader if not coord.is_self_hit()] #Remove self hits
    #take best alignment. We are defining the best alignment as the one with the best
    #seq identity and the highest aligned length, divided by the query seq length.
    #[S1]    [E1]    [S2]     [E2]     [LEN 1]   [LEN 2]   [% IDY]   [LEN R]   [LEN Q]   [COV R]   [COV Q]   [TAGS]
    cls_list = []
    best_class = None
    for a in alignments:
      a = str(a)
      print("a: ", a)
      fields = a.strip().split()  # Strip leading/trailing spaces and split by whitespace
      if len(fields) == 12:# Check if there are enough fields before creating a MUMmerDelta object
        cls = MUMmerDelta(int(fields[0]), int(fields[1]), int(fields[2]), int(fields[3]),
                        int(fields[4]), int(fields[5]), float(fields[6]), int(fields[7]),
                        int(fields[8]), int(fields[9]), str(fields[10]), str(fields[11]))
      else:
        print("Unexpected number of fields:", len(fields))
        raise NotImplementedError
      cls_list.append(cls)
      if best_class is None:
        best_class = cls
      elif cls.pct_idy*(abs(cls.s2-cls.e2)/klen) > best_class.pct_idy*(abs(best_class.s2-best_class.e2)/klen):
        best_class = cls
    #Take the best class and, if it is better than a threshold, take the coords of the
    #alignment and clip out the subsequence from the generead. Put that seq in the ret_dict
    if best_class != None and best_class.pct_idy > id_thresh and abs(best_class.s2-best_class.e2)/klen > lenpct_thresh:
      print("___________________")
      print("s1: ", best_class.s1)
      print("e1: ", best_class.e1)
      print("s2: ", best_class.s2)
      print("e2: ", best_class.e2)

      gseg = str(geneRead[1].seq)
      gseg = gseg[best_class.s1:best_class.e1]
      vv = str(k) + " gene from " + str(geneRead[1].description)
      print(vv)
      print("Gseg: ", gseg)
      if best_class.s2 < best_class.e2:    #forward
        sr = SeqRecord(seq=Seq(gseg), id=geneRead[0], description=vv)
      else:                                #reverse
        ir = Seq(gseg)
        inter = ir.reverse_complement()
        sr = SeqRecord(seq=inter, id=geneRead[0], description=vv)
      tuup = (geneRead[0], sr)
      ret_dict[k].append(tuup)
      best_classes[k] = best_class
    else:
      continue
    #clear classes in cls_list and then empty list
    for c in cls_list:
      del c
    cls_list = []

  #plot the pct id of each best_class in best_classes
  #ks = []
  #vals = []
  #for k in best_classes.keys():
  #  ks.append(k)
  #  vals.append(best_classes[k].pct_idy)
  #sns.barplot(x=ks, y=vals)
  #raise NotImplementedError
  return ret_dict

Make the directory if it isn't already there

In [ ]:
if not os.path.exists(dest_dir):
  os.mkdir(dest_dir)

#Plugin feature for later versions: isolate characterization into strains by external tool
strain_list = []
iso_list = []

Break gene dictionary into components and save into the file system

In [ ]:
def saver(gene_dict, dest = dest_dir):
  if not os.path.exists(dest):
    os.makedirs(dest)
  for gne in gene_dict.keys():
    if not gene_dict[gne]:
      continue
    os.makedirs(os.path.join(dest, gne), exist_ok=True)
    strains, seqs = zip(*[(x[0], x[1]) for x in gene_dict[gne]])
    assert len(strains) == len(seqs)
    #Need to run the aligner to snip out specific genes
    if gne == "CompleteGenomes" or gne == "UnlabeledGenomes":
      #continue
      #get new gene_dict by calling autoClassifier on the complete genome
      for tup in gene_dict[gne]:
        gd = autoClassifierPyMUMmer(tup, getGeneDict(), dest)
        saver(gd, dest)
    #Trust the label
    else:
      for i in range(len(strains)):
        strain = strains[i]
        seq = seqs[i]
        fin = strain + ".fasta"
        pth = os.path.join(dest, gne, fin)
        oneback = os.path.join(dest, gne)
        #If no file exists
        if not os.path.exists(oneback):
          os.mkdir(oneback)
          #Save seq to the gene/strain folder
          try:
            with open(pth, 'w') as f:
              SeqIO.write(seq, f, 'fasta')
          except AttributeError:
            seq.id = 1
            with open(pth, "w") as f:
              SeqIO.write(seq, f, 'fasta')
        #If one does, save to it
        else:
          try:
            with open(pth, "a") as f:
              SeqIO.write(seq, f, 'fasta')
          except AttributeError:
            seq.id = "1"
            with open(pth, "a") as f:
              SeqIO.write(seq, f, 'fasta')
      gene_dict[gne] = []

    try:
      for tup in gene_dict["UnlabledGenomes"]:
        gd = autoClassifierPyMUMmer(tup, getGeneDict(), dest)
        saver(gd, dest)
      gene_dict["UnlabledGenomes"] = []
    except KeyError:
      pass

    try:
      for tup in gene_dict["CompleteGenomes"]:
        gd = autoClassifierPyMUMmer(tup, getGeneDict(), dest)
        saver(gd, dest)
      gene_dict["CompleteGenomes"] = []
    except KeyError:
      pass



def getStrain(d: str, slist: list, isolist: list):
  if "STRAIN" in d.upper():
    before = d.upper().split("STRAIN")[0]
    after = d.upper().split("STRAIN")[1]
    try:
      if after[0] == " ":
        after = after[1:]
    except IndexError:
      return "UNKNOWN_STRAIN"
    if "," in after:
      ay = after.split(",")[0]
      az = ay.split(" ")[0]
      if az.upper() == "HSV1":
        az = ay.split(" ")[1]
        slist.append(az)
        return az
      else:
        slist.append(az)
        return az
    else:
      return after.split(" ")[0]

  elif "ISOLATE" in d.upper():
    before = d.upper().split("ISOLATE")[0]
    after = d.upper().split("ISOLATE")[1]
    try:
      if after[0] == " ":
        after = after[1:]
    except IndexError:
      return "UNKNOWN_STRAIN"
    if "," in after:
      ay = after.split(",")[0]
      az = ay.split(" ")[0]
      if az.upper() == "HSV1":
        az = ay.split(" ")[1]
        isolist.append(az)
        return az
      else:
        isolist.append(az)
        return az
    else:
      return after.split(" ")[0]

  else:
    return "UNKNOWN_STRAIN"


def cleanStrain(d: str):
  gtable = getHSV1GeneDict()
  for k in gtable.keys():
    if k in d:
      d = d.replace(k, "")
  if "/" in d:
    d = d.replace("/", "_")
  if " " in d:
    d = d.replace(" ", "_")
  d = d.replace(".", "_")
  if d[0] == "-":
    d = d[1:]
  if d[0] == "_":
    d = d[1:]
  if d[0] == ":":
    d = d[1:]
  if d == "":
    d = "UNKNOWN_STRAIN"
  if d[-1] == "_":
    d = d[:-1]
  if "(" in d:
    d = d.replace("(", "")
  if ")" in d:
    d = d.replace(")", "")
  if d == "":
    d = "UNKNOWN_STRAIN"
  return d

Process Program - Approach 1

In [ ]:
#Approach 1: Strain naiive / trust the label
#This can be refined by running a seperate script on the genomes directory and classifiying isolates into strain.

records = SeqIO.parse(genomes_dir)
#For each seqRecord in records, isolate strain and seq
for r in records:
  gene_table = {[x:[] for x in getGeneDict().keys()]}
  gene_table["CompleteGenomes"] = []
  gene_table["UnlabledGenomes"] = []

  cc = cleanStrain(getStrain(str(r.description), strain_list, iso_list))
  print(cc)

  if len(r.seq) > length_of_complete_genome:
    gene_table["CompleteGenomes"].append((cc,r))
    saver(gene_table, dest=dest_dir)
    continue
  swich = False
  for k in gene_table.keys():
    if k.upper() in r.description.upper():
      swich = True
      gene_table[k].append((cc,r))
      saver(gene_table, dest=dest_dir)
      #continue    #Don't continue because some records contain multiple genes
  if not swich:
    gene_table["UnlabledGenomes"].append((cc,r))
    saver(gene_table, dest=dest_dir)